# Анализ результатов A/B-тестирования

---

Необходимо провести анализ результатов A/B-тестирования, которое было запущено команией с целью увеличения дохода.

---

Исходные данные представлены в необработанном виде и содержат следующие столбцы:
* Дата совершения действия (**date_day**) – с точностью до дня;
* Наименование контрольной группы (**variant_group**), где «A1» и «A2»   - группы контрольного варианта, «B1» и «B2»  - группы тестового варианта;
* Количество пользователей (чел.) - **users**;
* Количество продаж (шт.) - **amount_sales**;
* Сумма, на которую были осуществлены продажи (у.е.) - **money_sales**.


---

In [ ]:
!pip install pymc -q

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import norm
from scipy.stats import variation
from scipy.stats import levene
from scipy.stats import kruskal
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import shapiro
import arviz
import plotly.express as px
import plotly.graph_objects as go
import pymc
import datetime

**<font size="6">Частотный подход</font>**

**<font size="4">Загрузка и обработка данных</font>**

Загрузка исходных данных

In [ ]:
path = '/kaggle/input/data-sales/data_sales.csv'
df = pd.read_csv(path)

In [ ]:
df.head()

**<font size="4">Статистика данных</font>**

Количество уникальных записей

In [ ]:
df.nunique()

Статистика

In [ ]:
df.describe()

Дисперсия дохода

In [ ]:
np.var(df['money_sales'])

**<font size="4">Обработка данных</font>**

Количество вариантов тестирования

In [ ]:
df['variant_group'].unique()

Удаление неопределённых вариантов

In [ ]:
df = df[df.variant_group != "none"]
df.reset_index(drop= True , inplace= True )

In [ ]:
df['variant_group'].unique()

In [ ]:
df.nunique()

Количество уникальных и неуникальных наборов (date_day + variant_group)

In [ ]:
unique_sets = df.groupby(['date_day', 'variant_group']).nunique()
total_rows = df.shape[0]
non_unique_sets = total_rows - unique_sets.shape[0]

print("Number of unique row sets:", unique_sets.shape[0])
print("Number of non-unique row sets:", non_unique_sets)

Удаление дубликатов

In [ ]:
df = df.groupby(['date_day', 'variant_group'], as_index=False).sum()

Количество уникальных записей после удаления дубликатов

In [ ]:
unique_sets = df.groupby(['date_day', 'variant_group']).nunique()
total_rows = df.shape[0]
non_unique_sets = total_rows - unique_sets.shape[0]

print("Number of unique row sets:", unique_sets.shape[0])
print("Number of non-unique row sets:", non_unique_sets)

In [ ]:
df.nunique()

In [ ]:
df.head(10)

Количество записей в группах после удаления дубликатов

In [ ]:
df["variant_group"].value_counts()

График распределения после обработки данных

In [ ]:
sns.boxplot(x = "variant_group", y = "money_sales", data = df);

Сортировка суммы, полученной от продаж в порядке убывания

In [ ]:
df.sort_values(by='money_sales', ascending=False).iloc[:10]

Сортировка количества продаж в порядке убывания

In [ ]:
df.sort_values(by='amount_sales', ascending=False).iloc[:10]

Сортировка количества пользователей в порядке убывания

In [ ]:
df.sort_values(by='users', ascending=False).iloc[:10]

Сортировка даты в порядке возрастания

In [ ]:
df.sort_values(by='date_day', ascending=True).iloc[:10]

Очистка данных от выбросов

In [ ]:
df['date_day'] = pd.to_datetime(df['date_day'])
df=df.loc[(df['date_day'] > '2021-12-31')]
df.reset_index(drop= True , inplace= True )
df.head(10)

In [ ]:
df.sort_values(by='date_day', ascending=True).iloc[:10]

Выбросы в данных не обнаружены

График распределения после обработки данных

In [ ]:
sns.boxplot(x = "variant_group", y = "money_sales", data = df);

Статистика после обработки данных

In [ ]:
df.groupby("variant_group")["money_sales"].describe()

Дисперсия дохода по группам

In [ ]:
dc1 = np.var(df.loc[df['variant_group'] == 'a1', 'money_sales'])
dv1 = np.var(df.loc[df['variant_group'] == 'b1', 'money_sales'])
dc2 = np.var(df.loc[df['variant_group'] == 'a2', 'money_sales'])
dv2 = np.var(df.loc[df['variant_group'] == 'b2', 'money_sales'])
print ('variance of control (A1) = %.5f, variance of control (A2) = %.5f' % (dc1,dc2))
print ('variance of variant (B1) = %.5f, variance of variant (B2) = %.5f' % (dv1,dv2))

In [ ]:
df.head(12)

**<font size="5">Статистический анализ</font>**

In [ ]:
df1=pd.DataFrame(df)
df1['variant_group'] = df1['variant_group'].str.replace('\d+', '')
df1

In [ ]:
unique_sets = df1.groupby(['date_day', 'variant_group']).nunique()
total_rows = df1.shape[0]
non_unique_sets = total_rows - unique_sets.shape[0]

print("Number of unique row sets:", unique_sets.shape[0])
print("Number of non-unique row sets:", non_unique_sets)

In [ ]:
df1 = df1.groupby(['date_day', 'variant_group'], as_index=False).sum()

In [ ]:
unique_sets = df1.groupby(['date_day', 'variant_group']).nunique()
total_rows = df1.shape[0]
non_unique_sets = total_rows - unique_sets.shape[0]

print("Number of unique row sets:", unique_sets.shape[0])
print("Number of non-unique row sets:", non_unique_sets)

In [ ]:
df1.head(25)

**<font size="4">Проверка распределения данных на нормальность</font>**

График распределения для всех пользователей

In [ ]:
f, ax = plt.subplots(1, figsize=(8,6))
sns.distplot(df1.loc[df1['variant_group'] == 'a', 'money_sales'], label='a')
sns.distplot(df1.loc[df1['variant_group'] == 'b', 'money_sales'], label='b')
plt.title('Distribution of revenue of all users')
plt.legend()
plt.subplots_adjust(hspace = 0.3)

Проверка статистических гипотез о нормальности распределения с помощью теста **Шапиро–Уилка**
* H0: Данные имеют нормальное распределение
* H1: Данные имеют ненормальное распределение

Результат теста Шапиро-Уилка для контрольной группы

In [ ]:
test_stat, pvalue = shapiro(df1.loc[df1["variant_group"] == "a", "money_sales"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Результат теста Шапиро-Уилка для вариантной группы

In [ ]:
test_stat, pvalue = shapiro(df1.loc[df1["variant_group"] == "b", "money_sales"])
print('Test Stat = %.5f, p-value = %.5f' % (test_stat, pvalue))

Поскольку значение p<=0,05, то нулевая гипотеза отвергается. Данные имеют ненормальное распределение

**<font size="4">Проверка совпадения дисперсий двух групп</font>**

Проверка статистических гипотез о совпадении дисперсий двух групп с помощью теста **Левена**

* H0: Вариантная и контрольная группы имеют одинаковую дисперсию 
* H1: Вариантная и контрольная группы не имеют одинаковую дисперсию

Результат теста Левена

In [ ]:
stats.levene(np.array([df1.loc[df1['variant_group'] == 'a', 'money_sales']]).flatten(),
             np.array([df1.loc[df1['variant_group'] == 'b', 'money_sales']]).flatten(), center='median')

Значение p>0,05, следовательно принимается нулевая гипотеза. Дисперсия обеих групп совпадает

**<font size="4">Проверка  присутствия значимой разницы между средним полученным доходом в обеих группах</font>**

Проверка статистических гипотез о присутствии значимой разницы между средним полученным доходом в контрольной и вариантной группе с помощью теста **Манна-Уитни**

* H0: Нет существенной разницы между средним доходом контрольной и вариантной групп
* H1: Существует значительная разница между средним доходом контрольной и вариантной групп

Результат теста Манна-Уитни

In [ ]:
test_stat, pvalue = mannwhitneyu(df1.loc[df1["variant_group"] == "a", "money_sales"],
                                 df1.loc[df1["variant_group"] == "b", "money_sales"])

print('Test Stat = %.5f, p-value = %.5f' % (test_stat, pvalue))

Значение p>0,05, поэтому можно принять нулевую гипотезу и сделать вывод, что нет существенной разницы между средним доходом контрольной и вариантной групп

**<font size="4">Исследование пользователей, приносящих доход</font>**

Очистка из набора данных пользователей с нулевым доходом

In [ ]:
df1_paying_user = df1.loc[df1["money_sales"] > 0]
df1_paying_user.reset_index(drop= True , inplace= True )

In [ ]:
df1_paying_user.head()

Статистика для доходных пользвателей

In [ ]:
df1_paying_user.groupby("variant_group")["money_sales"].describe()

Дисперсия дохода по группам для платящих пользователей

In [ ]:
dpc1 = np.var(df1_paying_user.loc[df1_paying_user['variant_group'] == 'a', 'money_sales'])
dpv1 = np.var(df1_paying_user.loc[df1_paying_user['variant_group'] == 'b', 'money_sales'])
print ('variance of control = %.5f, variance of variant = %.5f' % (dpc1,dpv1))

**<font size="4">Проверка распределения данных на нормальность</font>**

График распределения для доходных пользователей

In [ ]:
f, ax = plt.subplots(1, figsize=(8,6))
sns.distplot(df1.loc[(df1['variant_group'] == 'a') & (df1['money_sales'] > 0), 'money_sales'], label='a' )
sns.distplot(df1.loc[(df1['variant_group'] == 'b') & (df1['money_sales'] > 0), 'money_sales'], label='b' )
plt.title('Paying user revenue distribution')
plt.legend()
plt.subplots_adjust(hspace = 0.3)

Проверка статистических гипотез о нормальности распределения с помощью теста **Шапиро–Уилка**
* H0: Данные имеют нормальное распределение
* H1: Данные имеют ненормальное распределение

Результат теста Шапиро-Уилка для контрольной группы

In [ ]:
test_stat, pvalue = shapiro(df1_paying_user.loc[df1_paying_user["variant_group"] == "a", "money_sales"])
print('Test Stat = %.5f, p-value = %.5f' % (test_stat, pvalue))

Результат теста Шапиро-Уилка для вариантной группы

In [ ]:
test_stat, pvalue = shapiro(df1_paying_user.loc[df1_paying_user["variant_group"] == "b", "money_sales"])
print('Test Stat = %.5f, p-value = %.5f' % (test_stat, pvalue))

Поскольку значение p<=0,05, то нулевая гипотеза отвергается. Данные представляют собой ненормальное распределение

**<font size="4">Проверка совпадения дисперсий двух групп</font>**

Проверка статистических гипотез о совпадении дисперсий двух групп с помощью теста **Левена**

* H0: Вариантная и контрольная группы имеют одинаковую дисперсию 
* H1: Вариантная и контрольная группы не имеют одинаковую дисперсию

Результат теста Левена

In [ ]:
stats.levene(np.array([df1_paying_user.loc[df1_paying_user['variant_group'] == 'a', 'money_sales']]).flatten(),
             np.array([df1_paying_user.loc[df1_paying_user['variant_group'] == 'b', 'money_sales']]).flatten(), center='median')

Значение p>0,05, следовательно принимается нулевая гипотеза. Дисперсия обеих групп совпадает

**<font size="4">Проверка  присутствия значимой разницы между средним полученным доходом в обеих группах</font>**

Проверка статистических гипотез о присутствии значимой разницы между средним полученным доходом в контрольной и вариантной группе с помощью теста **Манна-Уитни**

* H0: Нет существенной разницы между средним доходом контрольной и вариантной групп
* H1: Существует значительная разница между средним доходом контрольной и вариантной групп

Результат теста Манна-Уитни

In [ ]:
test_stat, pvalue = mannwhitneyu(df1_paying_user.loc[df1["variant_group"] == "a", "money_sales"],
                                 df1_paying_user.loc[df1["variant_group"] == "b", "money_sales"])

print('Test Stat = %.5f, p-value = %.5f' % (test_stat, pvalue))

Значение p>0,05, поэтому нулевая гипотеза не отвергается и можно сделать вывод, что для пользователей только с положительным доходом также нет существенной разницы между средним доходом контрольной и вариантной групп

**<font size="4">ANOVA Test</font>**

* H0: Нет существенной разницы между средним доходом в группах
* H1: Существует значительная разница между средним доходом в группах

Результаты теста для среднего дохода:

In [ ]:
test_stat, pvalue = kruskal(df.loc[df["variant_group"] == 'a1', "money_sales"],
                                 df.loc[df["variant_group"] == 'a2', "money_sales"],
                                df.loc[df["variant_group"] == 'b1', "money_sales"],
                                df.loc[df["variant_group"] == 'b2', "money_sales"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Результаты теста для количества продаж:

In [ ]:
test_stat, pvalue = kruskal(df.loc[df["variant_group"] == 'a1', "amount_sales"],
                                 df.loc[df["variant_group"] == 'a2', "amount_sales"],
                                df.loc[df["variant_group"] == 'b1', "amount_sales"],
                                df.loc[df["variant_group"] == 'b2', "amount_sales"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

**<font size="6">Байесовский подход</font>**

Установка априорных значений для рассматриваемых параметров и запуск сэмплера

In [ ]:
group_idx, group_name = pd.factorize(df1["variant_group"])

with pymc.Model(coords={"groups": group_name}) as model:
    
    # Априорные значения
    p = pymc.TruncatedNormal(
        "p",
        mu=(df1["money_sales"] > 0).sum() / len(df1), # p-значение из данных (по обеим группам)
        sigma=0.1,
        lower=0,
        upper=1,
        dims="groups"
    )
    mu = pymc.TruncatedNormal(
        "mu",
        mu=df1.query("money_sales > 0")["money_sales"].mean(), # mu из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    sigma = pymc.TruncatedNormal("sigma",
        mu=df1.query("money_sales > 0")["money_sales"].std(ddof=1), # std из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    mean_difference = pymc.Deterministic("a - b"
                                         , mu[0] -  mu[1])
    # Вероятность соответствия
    pymc.Bernoulli(
        "p_obs",
        p=p[group_idx],
        observed=(df1["money_sales"] > 0).astype('float')
    )
    # Доход, приходящийся на одну продажу
    mask = df1["money_sales"] > 0
    y = pymc.Normal("y_obs", mu=mu[group_idx[mask]], sigma=sigma[group_idx[mask]], observed=df1.loc[mask,"money_sales"])

    # Запуск семплера
    trace = pymc.sample(1000, tune=1000)

Граифики распределений для обеих рассматриваемых групп

In [ ]:
arviz.plot_trace(trace, legend=True);
plt.tight_layout()

Графики 95% доверительных интервалов

In [ ]:
arviz.plot_posterior(trace, grid=(4,2), hdi_prob=0.95);
plt.tight_layout()

In [ ]:
pa = trace.posterior.sel(groups="a")["p"]
pb = trace.posterior.sel(groups="b")["p"]
mua = trace.posterior.sel(groups="a")["mu"]
mub = trace.posterior.sel(groups="b")["mu"]

# Differences
mudiff1 = (mua - mub).values.reshape(-1)
pdiff1 = (pa - pb).values.reshape(-1)
mudiff2 = (mub - mua).values.reshape(-1)
pdiff2 = (pb - pa).values.reshape(-1)
print("Вероятность того, что группа A имеет большую вероятность успеха: ", (pdiff1 > 0).mean())
print("Вероятность того, что группа A имеет большее среднее значение дохода: ", (mudiff1 > 0).mean())
print("-----------------------------------------------------------------------------")
print("Вероятность того, что группа B имеет большую вероятность успеха: ", (pdiff2 > 0).mean())
print("Вероятность того, что группа B имеет большее среднее значение дохода: ", (mudiff2 > 0).mean())

**<font size="5">A1/B1</font>**

In [ ]:
df2=pd.DataFrame(df)
df2

In [ ]:
var = ['a1', 'b1']
df2=df2.loc[df2['variant_group'].isin(var)]
df2.reset_index(drop= True , inplace= True)
df2

In [ ]:
group_idx, group_name = pd.factorize(df2["variant_group"])

with pymc.Model(coords={"groups": group_name}) as model:
    
    # Априорные значения
    p = pymc.TruncatedNormal(
        "p",
        mu=(df2["money_sales"] > 0).sum() / len(df2), # p-значение из данных (по обеим группам)
        sigma=0.1,
        lower=0,
        upper=1,
        dims="groups"
    )
    mu = pymc.TruncatedNormal(
        "mu",
        mu=df2.query("money_sales > 0")["money_sales"].mean(), # mu из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    sigma = pymc.TruncatedNormal("sigma",
        mu=df2.query("money_sales > 0")["money_sales"].std(ddof=1), # std из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    mean_difference = pymc.Deterministic("a - b"
                                         , mu[0] -  mu[1])
    # Вероятность соответствия
    pymc.Bernoulli(
        "p_obs",
        p=p[group_idx],
        observed=(df2["money_sales"] > 0).astype('float')
    )
    # Доход, приходящийся на одну продажу
    mask = df2["money_sales"] > 0
    y = pymc.Normal("y_obs", mu=mu[group_idx[mask]], sigma=sigma[group_idx[mask]], observed=df2.loc[mask,"money_sales"])

    # Запуск семплера
    trace = pymc.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(trace, legend=True);
plt.tight_layout()

In [ ]:
arviz.plot_posterior(trace, grid=(4,2), hdi_prob=0.95);
plt.tight_layout()

In [ ]:
pa = trace.posterior.sel(groups="a1")["p"]
pb = trace.posterior.sel(groups="b1")["p"]
mua = trace.posterior.sel(groups="a1")["mu"]
mub = trace.posterior.sel(groups="b1")["mu"]

# Differences
mudiff1 = (mua - mub).values.reshape(-1)
pdiff1 = (pa - pb).values.reshape(-1)
mudiff2 = (mub - mua).values.reshape(-1)
pdiff2 = (pb - pa).values.reshape(-1)
print("Вероятность того, что группа A имеет большую вероятность успеха: ", (pdiff1 > 0).mean())
print("Вероятность того, что группа A имеет большее среднее значение дохода: ", (mudiff1 > 0).mean())
print("-----------------------------------------------------------------------------")
print("Вероятность того, что группа B имеет большую вероятность успеха: ", (pdiff2 > 0).mean())
print("Вероятность того, что группа B имеет большее среднее значение дохода: ", (mudiff2 > 0).mean())

**<font size="5">A2/B2</font>**

In [ ]:
df3=pd.DataFrame(df)
df3

In [ ]:
var1 = ['a2', 'b2']
df3=df3.loc[df3['variant_group'].isin(var1)]
df3.reset_index(drop= True , inplace= True)
df3

In [ ]:
group_idx, group_name = pd.factorize(df3["variant_group"])

with pymc.Model(coords={"groups": group_name}) as model:
    
    # Априорные значения
    p = pymc.TruncatedNormal(
        "p",
        mu=(df3["money_sales"] > 0).sum() / len(df3), # p-значение из данных (по обеим группам)
        sigma=0.1,
        lower=0,
        upper=1,
        dims="groups"
    )
    mu = pymc.TruncatedNormal(
        "mu",
        mu=df3.query("money_sales > 0")["money_sales"].mean(), # mu из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    sigma = pymc.TruncatedNormal("sigma",
        mu=df3.query("money_sales > 0")["money_sales"].std(ddof=1), # std из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    mean_difference = pymc.Deterministic("a - b"
                                         , mu[0] -  mu[1])
    # Вероятность соответствия
    pymc.Bernoulli(
        "p_obs",
        p=p[group_idx],
        observed=(df3["money_sales"] > 0).astype('float')
    )
    # Доход, приходящийся на одну продажу
    mask = df3["money_sales"] > 0
    y = pymc.Normal("y_obs", mu=mu[group_idx[mask]], sigma=sigma[group_idx[mask]], observed=df3.loc[mask,"money_sales"])

    # Запуск семплера
    trace = pymc.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(trace, legend=True);
plt.tight_layout()

In [ ]:
arviz.plot_posterior(trace, grid=(4,2), hdi_prob=0.95);
plt.tight_layout()

In [ ]:
pa = trace.posterior.sel(groups="a2")["p"]
pb = trace.posterior.sel(groups="b2")["p"]
mua = trace.posterior.sel(groups="a2")["mu"]
mub = trace.posterior.sel(groups="b2")["mu"]

# Differences
mudiff1 = (mua - mub).values.reshape(-1)
pdiff1 = (pa - pb).values.reshape(-1)
mudiff2 = (mub - mua).values.reshape(-1)
pdiff2 = (pb - pa).values.reshape(-1)
print("Вероятность того, что группа A имеет большую вероятность успеха: ", (pdiff1 > 0).mean())
print("Вероятность того, что группа A имеет большее среднее значение дохода: ", (mudiff1 > 0).mean())
print("-----------------------------------------------------------------------------")
print("Вероятность того, что группа B имеет большую вероятность успеха: ", (pdiff2 > 0).mean())
print("Вероятность того, что группа B имеет большее среднее значение дохода: ", (mudiff2 > 0).mean())

**<font size="5">A1/A2</font>**

In [ ]:
df4=pd.DataFrame(df)
df4

In [ ]:
var2 = ['a1', 'a2']
df4=df4.loc[df4['variant_group'].isin(var2)]
df4.reset_index(drop= True , inplace= True)
df4

In [ ]:
group_idx, group_name = pd.factorize(df4["variant_group"])

with pymc.Model(coords={"groups": group_name}) as model:
    
    # Априорные значения
    p = pymc.TruncatedNormal(
        "p",
        mu=(df4["money_sales"] > 0).sum() / len(df4), # p-значение из данных (по обеим группам)
        sigma=0.1,
        lower=0,
        upper=1,
        dims="groups"
    )
    mu = pymc.TruncatedNormal(
        "mu",
        mu=df4.query("money_sales > 0")["money_sales"].mean(), # mu из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    sigma = pymc.TruncatedNormal("sigma",
        mu=df4.query("money_sales > 0")["money_sales"].std(ddof=1), # std из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    mean_difference = pymc.Deterministic("a1 - a2"
                                         , mu[0] -  mu[1])
    # Вероятность соответствия
    pymc.Bernoulli(
        "p_obs",
        p=p[group_idx],
        observed=(df4["money_sales"] > 0).astype('float')
    )
    # Доход, приходящийся на одну продажу
    mask = df4["money_sales"] > 0
    y = pymc.Normal("y_obs", mu=mu[group_idx[mask]], sigma=sigma[group_idx[mask]], observed=df4.loc[mask,"money_sales"])

    # Запуск семплера
    trace = pymc.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(trace, legend=True);
plt.tight_layout()

In [ ]:
arviz.plot_posterior(trace, grid=(4,2), hdi_prob=0.95);
plt.tight_layout()

In [ ]:
pa = trace.posterior.sel(groups="a1")["p"]
pb = trace.posterior.sel(groups="a2")["p"]
mua = trace.posterior.sel(groups="a1")["mu"]
mub = trace.posterior.sel(groups="a2")["mu"]

# Differences
mudiff1 = (mua - mub).values.reshape(-1)
pdiff1 = (pa - pb).values.reshape(-1)
mudiff2 = (mub - mua).values.reshape(-1)
pdiff2 = (pb - pa).values.reshape(-1)
print("Вероятность того, что группа A имеет большую вероятность успеха: ", (pdiff1 > 0).mean())
print("Вероятность того, что группа A имеет большее среднее значение дохода: ", (mudiff1 > 0).mean())
print("-----------------------------------------------------------------------------")
print("Вероятность того, что группа B имеет большую вероятность успеха: ", (pdiff2 > 0).mean())
print("Вероятность того, что группа B имеет большее среднее значение дохода: ", (mudiff2 > 0).mean())

**<font size="5">B1/B2</font>**

In [ ]:
df5=pd.DataFrame(df)
df5

In [ ]:
var3 = ['b1', 'b2']
df5=df5.loc[df5['variant_group'].isin(var3)]
df5.reset_index(drop= True , inplace= True)
df5

In [ ]:
group_idx, group_name = pd.factorize(df5["variant_group"])

with pymc.Model(coords={"groups": group_name}) as model:
    
    # Априорные значения
    p = pymc.TruncatedNormal(
        "p",
        mu=(df5["money_sales"] > 0).sum() / len(df4), # p-значение из данных (по обеим группам)
        sigma=0.1,
        lower=0,
        upper=1,
        dims="groups"
    )
    mu = pymc.TruncatedNormal(
        "mu",
        mu=df5.query("money_sales > 0")["money_sales"].mean(), # mu из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    sigma = pymc.TruncatedNormal("sigma",
        mu=df5.query("money_sales > 0")["money_sales"].std(ddof=1), # std из набора данных (по обеим группам)
        sigma=10,
        lower=0,
        dims="groups"
    )
    mean_difference = pymc.Deterministic("b1 - b2"
                                         , mu[0] -  mu[1])
    # Вероятность соответствия
    pymc.Bernoulli(
        "p_obs",
        p=p[group_idx],
        observed=(df5["money_sales"] > 0).astype('float')
    )
    # Доход, приходящийся на одну продажу
    mask = df5["money_sales"] > 0
    y = pymc.Normal("y_obs", mu=mu[group_idx[mask]], sigma=sigma[group_idx[mask]], observed=df5.loc[mask,"money_sales"])

    # Запуск семплера
    trace = pymc.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(trace, legend=True);
plt.tight_layout()

In [ ]:
arviz.plot_posterior(trace, grid=(4,2), hdi_prob=0.95);
plt.tight_layout()

## Выводы

Проведя анализ с помощью обоих статистических подходов, можно сказать, что частотный и байесовский подходы привели к похожим заключениям.

Результаты анализа частотным подходом показали отсутствие статистически значимой разницы в среднем доходе между группами. То же самое было проделано для данных, где доход имел только положительные значения, и результаты получились аналогичными. Эти результаты подтвердило и множественное сравнение групп между собой.

Байесовский подход показал, в целом, похожий результат. Разница в среднем доходе не является значительной и наблюдается условно. Также был подтверждён вывод о том, что группа B имеет немного более высокую вероятность успеха, чем группа A. Опять же, разница между ними не является значительной. Вероятность успеха продажи в обеих группах достаточно близка, что может указывать на схожесть групп в этом аспекте.

Полученные вероятностные оценки подтвердили вывод о том, что группа несколько A превосходит группу B по показателю среднего дохода, а группа B несколько превосходит группа A по показателю вероятности успешной продажи. Однако данное превосходство не является значительным.

Результаты сравнения подгрупп между собой показали похожий результат. Статистически значимые преимущества какой-либо из групп по каждому из параметров отсутствуют.